<a href="https://colab.research.google.com/github/sanamsabooni/App_HW/blob/main/Connect_to_APIs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Install Required Packages

In [37]:
!pip install requests pandas python-dotenv

Set Up Folder for Data Storage

In [38]:
import os

save_folder = r"C:\ZohoData"

# Create the folder if it doesn't exist
if not os.path.exists(save_folder):
    os.makedirs(save_folder)
    print(f"Folder '{save_folder}' created successfully.")
else:
    print(f"Folder '{save_folder}' already exists.")


Folder 'C:\ZohoData' already exists.


Set Up Zoho API Credentials

Create a .env file in the same directory where your script is located and store your Zoho API credentials securely:

.env file content:



Function to Refresh Access Token

In [39]:
import requests
import json
from dotenv import load_dotenv
import os

# Load environment variables from the .env file
load_dotenv("hw.env")

# Retrieve credentials from the environment
CLIENT_ID = os.getenv('CLIENT_ID')
CLIENT_SECRET = os.getenv('CLIENT_SECRET')
REFRESH_TOKEN = os.getenv('REFRESH_TOKEN')
TOKEN_URL = 'https://accounts.zoho.com/oauth/v2/token'

# Function to refresh the access token
def refresh_access_token():
    data = {
        'refresh_token': REFRESH_TOKEN,
        'client_id': CLIENT_ID,
        'client_secret': CLIENT_SECRET,
        'grant_type': 'refresh_token'
    }

    response = requests.post(TOKEN_URL, data=data)
    if response.status_code == 200:
        access_token = response.json().get('access_token')
        print("New Access Token Retrieved.")

        # Save the access token to a local file
        token_file_path = os.path.join(r"C:\ZohoData", "access_token.json")
        with open(token_file_path, "w") as token_file:
            json.dump({"access_token": access_token}, token_file, indent=4)

        print(f"Access token saved to {token_file_path}")
        return access_token
    else:
        print("Failed to refresh access token:", response.json())
        return None


Function to Read Saved Access Token

In [40]:
def get_access_token():
    token_file_path = os.path.join(r"C:\ZohoData", "access_token.json")

    if os.path.exists(token_file_path):
        with open(token_file_path, "r") as token_file:
            token_data = json.load(token_file)
            return token_data.get("access_token")
    else:
        print("Access token file not found. Refreshing token...")
        return refresh_access_token()


Fetch Zoho CRM Modules and Save Locally

In [41]:
def get_modules():
    access_token = get_access_token()
    modules_url = "https://www.zohoapis.com/crm/v2/settings/modules"
    headers = {
        'Authorization': f'Bearer {access_token}'
    }

    response = requests.get(modules_url, headers=headers)

    if response.status_code == 200:
        modules = response.json().get('modules', [])

        # Extract module names and IDs
        module_details = [{"name": mod["api_name"], "id": mod["id"]} for mod in modules]

        # Save modules to a file
        modules_file_path = os.path.join(r"C:\ZohoData", "zoho_modules.json")
        with open(modules_file_path, "w") as f:
            json.dump(module_details, f, indent=4)

        print(f"Modules saved to {modules_file_path}")
        return module_details
    else:
        print("Error fetching modules:", response.json())
        return None


Verify the Saved Data

In [42]:
def check_saved_data():
    token_path = os.path.join(r"C:\ZohoData", "access_token.json")
    modules_path = os.path.join(r"C:\ZohoData", "zoho_modules.json")

    if os.path.exists(token_path):
        print("✅ Access token file exists.")
    else:
        print("❌ Access token file is missing.")

    if os.path.exists(modules_path):
        print("✅ Modules file exists.")
        with open(modules_path, "r") as f:
            data = json.load(f)
            print(f"Total Modules Retrieved: {len(data)}")
    else:
        print("❌ Modules file is missing.")


Run the Full Workflow

In [43]:
# Step 1: Refresh token and store locally
refresh_access_token()

# Step 2: Fetch modules and store locally
get_modules()

# Step 3: Check if files were saved correctly
check_saved_data()


New Access Token Retrieved.
Access token saved to C:\ZohoData/access_token.json
Error fetching modules: {'code': 'INVALID_TOKEN', 'details': {}, 'message': 'invalid oauth token', 'status': 'error'}
✅ Access token file exists.
❌ Modules file is missing.


Validate the Saved Access Token

In [47]:
def validate_access_token():
    access_token = get_access_token()  # Load the token from file
    test_url = "https://www.zohoapis.com/crm/v2/settings/modules"  # Replace with correct domain for your region
    headers = {"Authorization": f"Bearer {access_token}"}

    response = requests.get(test_url, headers=headers)
    print(f"Validation Status Code: {response.status_code}")
    print(f"Response: {response.json()}")


Debug the Token Refresh Process

In [48]:
def refresh_access_token():
    data = {
        'refresh_token': REFRESH_TOKEN,
        'client_id': CLIENT_ID,
        'client_secret': CLIENT_SECRET,
        'grant_type': 'refresh_token'
    }

    response = requests.post(TOKEN_URL, data=data)
    print(f"Refresh Response: {response.json()}")  # Debugging line
    if response.status_code == 200:
        access_token = response.json().get('access_token')
        print("New Access Token Retrieved.")

        # Save the access token to a local file
        token_file_path = os.path.join(r"C:\ZohoData", "access_token.json")
        with open(token_file_path, "w") as token_file:
            json.dump({"access_token": access_token}, token_file, indent=4)

        print(f"Access token saved to {token_file_path}")
        return access_token
    else:
        print("Failed to refresh access token:", response.json())
        return None


Ensure Correct Scopes Are Enabled

Automate Token Refresh (Optional)

In [44]:
import schedule
import time

def scheduled_token_refresh():
    refresh_access_token()
    print("Token refreshed successfully.")

# Schedule the task to run every 24 hours
schedule.every().day.at("01:00").do(scheduled_token_refresh)

while True:
    schedule.run_pending()
    time.sleep(60)  # Check every minute


KeyboardInterrupt: 

In [15]:
!pip install requests #Requests: To handle HTTP requests, including making API calls to Zoho CRM.
!pip install pandas #Pandas (Optional but recommended): If you want to work with tabular data (like displaying modules in a table), Pandas makes this much easier.
!pip install schedule #Schedule (For scheduling automatic token renewal): This allows you to run the token renewal process periodically (daily or weekly).
!pip install python-dotenv #(Python-dotenv (Optional for environment variables): If you want to keep your credentials (like client_id, client_secret, refresh_token) in a .env file for better security.)

In [16]:
# import needed packages and libraries

import requests
import json
import pandas as pd
import schedule
import time
from dotenv import load_dotenv
import os

**save the retrieved Zoho CRM data in the C:\ZohoData folder.**

In [17]:
def get_access_token():
    token_path = os.path.join(save_folder, "access_token.json")

    if not os.path.exists(token_path):
        print("Token file not found. Please refresh the token first.")
        return None

    with open(token_path, "r") as token_file:
        return json.load(token_file).get("access_token")


In [18]:
{
    "access_token": "YOUR_VALID_ACCESS_TOKEN"
}


{'access_token': 'YOUR_VALID_ACCESS_TOKEN'}

In [19]:
import time

def refresh_access_token_with_retry():
    retry_attempts = 5
    wait_time = 60  # Start with 60 seconds wait

    for attempt in range(retry_attempts):
        access_token = refresh_access_token()
        if access_token:
            return access_token
        print(f"Retrying in {wait_time} seconds...")
        time.sleep(wait_time)
        wait_time *= 2  # Double the wait time with each retry

    print("Max retry attempts reached. Try again later.")
    return None


In [20]:
import requests
import json
import os

# Define folder path to save the data
save_folder = r"C:\ZohoData"
os.makedirs(save_folder, exist_ok=True)

# Your Zoho API credentials
CLIENT_ID = 'YOUR_CLIENT_ID'
CLIENT_SECRET = 'YOUR_CLIENT_SECRET'
REFRESH_TOKEN = 'YOUR_REFRESH_TOKEN'
TOKEN_URL = 'https://accounts.zoho.com/oauth/v2/token'

def refresh_access_token():
    data = {
        'refresh_token': REFRESH_TOKEN,
        'client_id': CLIENT_ID,
        'client_secret': CLIENT_SECRET,
        'grant_type': 'refresh_token'
    }

    response = requests.post(TOKEN_URL, data=data)
    if response.status_code == 200:
        access_token = response.json().get('access_token')

        # Save the token locally
        token_path = os.path.join(save_folder, "access_token.json")
        with open(token_path, "w") as token_file:
            json.dump({"access_token": access_token}, token_file)

        print("Access token refreshed and saved locally.")
        return access_token
    else:
        print("Failed to refresh access token:", response.json())
        return None

# Refresh and store the token
access_token = refresh_access_token()

# API URL to fetch modules
MODULES_URL = "https://www.zohoapis.com/crm/v2/settings/modules"

def get_modules():
    token_path = os.path.join(save_folder, "access_token.json")
    with open(token_path, "r") as token_file:
        access_token = json.load(token_file)["access_token"]

    headers = {
        'Authorization': f'Bearer {access_token}'
    }

    response = requests.get(MODULES_URL, headers=headers)

    if response.status_code == 200:
        modules = response.json().get('modules', [])

        # Save modules data locally
        modules_path = os.path.join(save_folder, "zoho_modules.json")
        with open(modules_path, "w") as f:
            json.dump(modules, f, indent=4)

        print(f"Modules saved to {modules_path}")
        return modules
    else:
        print("Error fetching modules:", response.json())
        return None

# Fetch and store modules locally
get_modules()


Failed to refresh access token: {'error_description': 'You have made too many requests continuously. Please try again after some time.', 'error': 'Access Denied', 'status': 'failure'}
Error fetching modules: {'code': 'INVALID_TOKEN', 'details': {}, 'message': 'invalid oauth token', 'status': 'error'}


In [21]:
import os
import json

# Define folder path
save_folder = r"C:\ZohoData"

# Ensure the folder exists
if not os.path.exists(save_folder):
    os.makedirs(save_folder)

# Sample data to save
sample_data = {"message": "Test file saved successfully"}

# Save the test file
test_file_path = os.path.join(save_folder, "test_file.json")
with open(test_file_path, "w") as file:
    json.dump(sample_data, file, indent=4)

print(f"Test file saved at: {test_file_path}")


Test file saved at: C:\ZohoData/test_file.json


**Token Renewal:**
 We can do it once a day now.
running the following code will do that for us and it is valid only for an hour.

**Manually Refresh the Token**

In [ ]:
def refresh_zoho_token():
    # Define your refresh token and client credentials
    refresh_token = 'YOUR_REFRESH_TOKEN'
    client_id = 'YOUR_CLIENT_ID'
    client_secret = 'YOUR_CLIENT_SECRET'

    # API URL to refresh the token
    url = 'https://accounts.zoho.com/oauth/v2/token'

    # Payload for the POST request
    payload = {
        'refresh_token': refresh_token,
        'client_id': client_id,
        'client_secret': client_secret,
        'grant_type': 'refresh_token'
    }

    # Send POST request to refresh the token
    response = requests.post(url, data=payload)

    # If the request was successful, parse and store the new access token
    if response.status_code == 200:
        data = response.json()
        access_token = data.get('access_token')
        print(f'New Access Token: {access_token}')
        return access_token
    else:
        print(f'Failed to refresh token: {response.status_code}')
        return None

# Call the function to refresh the token
refresh_zoho_token()


New Access Token: None


In [ ]:
import requests
import json

In [ ]:
# Your Zoho API credentials
CLIENT_ID = '1000.YMECBOG5M2VOMPFU0FJI5YED0JX3XX'
CLIENT_SECRET = '0747da4549e31db730835243ffe9008c58514af154'
REFRESH_TOKEN = '1000.729179886aae591f83b312ad058916c4.7c87cdc83e04efc88e9b013825ba28d4'
TOKEN_URL = 'https://accounts.zoho.com/oauth/v2/token'

def refresh_access_token():
    data = {
        'refresh_token': REFRESH_TOKEN,
        'client_id': CLIENT_ID,
        'client_secret': CLIENT_SECRET,
        'grant_type': 'refresh_token'
    }

    response = requests.post(TOKEN_URL, data=data)
    if response.status_code == 200:
        access_token = response.json().get('access_token')
        print("New Access Token:", access_token)

        # Save the token to a file for future use
        with open("access_token.json", "w") as token_file:
            json.dump({"access_token": access_token}, token_file)

        return access_token
    else:
        print("Failed to refresh access token:", response.json())
        return None

# Refresh the token and store it
access_token = refresh_access_token()


New Access Token: 1000.e405f2293fd87a9508349a09d959bcbf.71f39f05661f3c6b570709702d6b4fab


 **Retrieve Modules and Extract**

In [ ]:
# Load the access token from the saved file
def get_access_token():
    with open("access_token.json", "r") as token_file:
        token_data = json.load(token_file)
        return token_data["access_token"]

# API URL to fetch modules
MODULES_URL = "https://www.zohoapis.com/crm/v2/settings/modules"

def get_modules():
    access_token = get_access_token()
    headers = {
        'Authorization': f'Bearer {access_token}'
    }

    response = requests.get(MODULES_URL, headers=headers)

    if response.status_code == 200:
        modules = response.json().get('modules', [])

        # Extract module names and IDs
        module_details = [{"name": mod["api_name"], "id": mod["id"]} for mod in modules]

        print("Retrieved Modules:")
        for module in module_details:
            print(f"Module Name: {module['name']}, ID: {module['id']}")

        # Save modules to a file
        with open("zoho_modules.json", "w") as f:
            json.dump(module_details, f, indent=4)

        return module_details
    else:
        print("Error fetching modules:", response.json())
        return None

# Fetch modules from Zoho CRM
get_modules()


Retrieved Modules:
Module Name: Feeds, ID: 525504000007868001
Module Name: CustomModule5001, ID: 525504000077920803
Module Name: SalesInbox, ID: 525504000018145001
Module Name: Home, ID: 525504000000002173
Module Name: Accounts, ID: 525504000000002177
Module Name: Sales_Orders, ID: 525504000000002221
Module Name: Invoices, ID: 525504000000002225
Module Name: Cancellations, ID: 525504000005752935
Module Name: ACH_Reject, ID: 525504000005832557
Module Name: Cases, ID: 525504000000002209
Module Name: Account_Analysis, ID: 525504000005894240
Module Name: Tin_Invalid, ID: 525504000005832158
Module Name: Reports, ID: 525504000000002185
Module Name: Products, ID: 525504000000002213
Module Name: Notes, ID: 525504000000002197
Module Name: Documents, ID: 525504000002936001
Module Name: Solutions, ID: 525504000000002211
Module Name: Purchase_Orders, ID: 525504000000002223
Module Name: Leads, ID: 525504000000002175
Module Name: Collections, ID: 525504000008251239
Module Name: Status_History, ID: 5

[{'name': 'Feeds', 'id': '525504000007868001'},
 {'name': 'CustomModule5001', 'id': '525504000077920803'},
 {'name': 'SalesInbox', 'id': '525504000018145001'},
 {'name': 'Home', 'id': '525504000000002173'},
 {'name': 'Accounts', 'id': '525504000000002177'},
 {'name': 'Sales_Orders', 'id': '525504000000002221'},
 {'name': 'Invoices', 'id': '525504000000002225'},
 {'name': 'Cancellations', 'id': '525504000005752935'},
 {'name': 'ACH_Reject', 'id': '525504000005832557'},
 {'name': 'Cases', 'id': '525504000000002209'},
 {'name': 'Account_Analysis', 'id': '525504000005894240'},
 {'name': 'Tin_Invalid', 'id': '525504000005832158'},
 {'name': 'Reports', 'id': '525504000000002185'},
 {'name': 'Products', 'id': '525504000000002213'},
 {'name': 'Notes', 'id': '525504000000002197'},
 {'name': 'Documents', 'id': '525504000002936001'},
 {'name': 'Solutions', 'id': '525504000000002211'},
 {'name': 'Purchase_Orders', 'id': '525504000000002223'},
 {'name': 'Leads', 'id': '525504000000002175'},
 {'name

In [ ]:
import requests
import pandas as pd

def get_zoho_modules():
    # Define your access token and Zoho API endpoint
    access_token = 'YOUR_ACCESS_TOKEN'  # Replace with your valid access token
    url = "https://www.zohoapis.com/crm/v2/settings/modules"

    # Headers to include access token in request
    headers = {
        "Authorization": f"Bearer {access_token}"
    }

    # Send the GET request to fetch the modules
    response = requests.get(url, headers=headers)

    # Check if the request was successful
    if response.status_code == 200:
        modules_data = response.json().get('modules', [])
        print("Module Name | Fields")
        print("-" * 50)

        # Iterate through the modules and print their name and fields
        for module in modules_data:
            module_name = module.get('api_name', 'N/A')
            fields = module.get('fields', [])
            field_names = [field.get('api_name', 'N/A') for field in fields]

            # Print module name and the fields underneath it
            print(f"\n{module_name}:")
            print("\t", ", ".join(field_names))

            # Optionally, you can store the result in a DataFrame for easier view or further processing
            df = pd.DataFrame([{'Module Name': module_name, 'Fields': ", ".join(field_names)}])
            print(df.to_string(index=False))
    else:
        print(f"Failed to fetch modules: {response.status_code}")

# Call the function to display the modules and their fields
get_zoho_modules()


Failed to fetch modules: 401


In [ ]:
!pip install requests prettytable


In [ ]:
import requests
from prettytable import PrettyTable

def fetch_zoho_modules():
    # Define your Zoho CRM API credentials
    access_token = 'YOUR_ACCESS_TOKEN'  # Replace with your valid access token

    # Zoho CRM API endpoint to fetch modules
    url = 'https://www.zohoapis.com/crm/v2/settings/modules'

    # Headers with the authorization token
    headers = {
        'Authorization': f'Zoho-oauthtoken {access_token}'
    }

    # Send GET request to fetch the modules
    response = requests.get(url, headers=headers)

    # Check if the request was successful
    if response.status_code == 200:
        data = response.json()
        modules = data.get('modules', [])

        # Create a PrettyTable instance to format the output
        table = PrettyTable()
        table.field_names = ["Module Name", "API Name", "Is Creatable", "Is Editable", "Is Viewable"]

        # Loop through the modules and add them to the table
        for module in modules:
            module_name = module.get('module_name')
            api_name = module.get('api_name')
            is_creatable = module.get('creatable')
            is_editable = module.get('editable')
            is_viewable = module.get('viewable')

            # Add the module details to the table
            table.add_row([module_name, api_name, is_creatable, is_editable, is_viewable])

        # Print the table
        print(table)

    else:
        print(f"Failed to fetch modules. Status code: {response.status_code}")
        print(response.text)

# Call the function to fetch and display the modules
fetch_zoho_modules()


Failed to fetch modules. Status code: 401
{"code":"INVALID_TOKEN","details":{},"message":"invalid oauth token","status":"error"}

